# Loading Data and Preprocessing

In [ ]:
%%capture
%run "preprocessing.ipynb"

In [ ]:
df_proc = df_proc # type: ignore

# RFMVD Score

R(Recenecy): How recently did the customer purchase?

F(Frequency): How often do they purchase?

M(Monetary): How much do they spend?

V(Variety): How many products sold by each customer?

D(Duration): How much time between each two transactions?

## RFM

Recency


In [ ]:
# In old code, this was formatted as ,format='%d-%m-%Y %H:%M' [REVISE]
max_date = df_proc['InvoiceDate'].max()
df_proc['Diff'] = max_date - df_proc['InvoiceDate']

# Last transaction date for each customer
rfm_r = df_proc.groupby('Customer ID')['Diff'].min().reset_index()
rfm_r['Diff'] = rfm_r['Diff'].dt.days
rfm_r.columns =  ['Customer ID', 'Recency']
rfm_r.head(1)

,Customer ID,Recency
0,12346.0,528


In [ ]:
# Frequency
rfm_f = df_proc.groupby('Customer ID')['Invoice'].count().reset_index()
rfm_f.columns = ['Customer ID', 'Frequency']
rfm_f.head(1)

,Customer ID,Frequency
0,12346.0,33


In [ ]:
# Monetary 
df_proc['Total'] = df_proc['Price'] * df_proc['Quantity']
rfm_m = df_proc.groupby('Customer ID')['Total'].sum().reset_index()
rfm_m.columns = ['Customer ID', 'Monetary']
rfm_m.head(1)

,Customer ID,Monetary
0,12346.0,372.86


In [ ]:
# Creating the RFM data frame.
rfm = pd.merge(rfm_r, rfm_f, on='Customer ID', how='inner')
rfm = pd.merge(rfm, rfm_m, on='Customer ID', how='inner')
rfm.head(1)

,Customer ID,Recency,Frequency,Monetary
0,12346.0,528,33,372.86


## VD

Variety

In [ ]:
def div(x: pd.Series):
    u = x.nunique()
    return u

rfm_v = df_proc.groupby('Customer ID')['StockCode'].agg([div]).reset_index()
rfm_v.columns = ['Customer ID', 'Variety']
rfm_v.head(1)

,Customer ID,Variety
0,12346.0,26


In [ ]:
# Duration
import statistics

def duration(x: pd.Series):
    m = []
    x = pd.to_datetime(x, format='%d-%m-%Y %H:%M')
    for i in range(1, len(x)-1):
        diff =  x.iloc[i] - x.iloc[i - 1]
        m.append(diff.days)

    return statistics.mean(m) if m else 0 # Should be revised how to handle empty arrays

rfm_d= df_proc.groupby('Customer ID')['InvoiceDate'].agg([duration]).reset_index()
rfm_d.columns = ['Customer ID', 'Duration']
rfm_d.head(1)


,Customer ID,Duration
0,12346.0,6.193548


In [ ]:
# Ceating the RFMVD data frame.
vd = pd.merge(rfm_v, rfm_d, on='Customer ID', how='inner')
rfmvd = pd.merge(rfm, vd, on='Customer ID', how='inner')
rfmvd.head(1)

,Customer ID,Recency,Frequency,Monetary,Variety,Duration
0,12346.0,528,33,372.86,26,6.193548


In [ ]:
rfmvd.corr()

,Customer ID,Recency,Frequency,Monetary,Variety,Duration
Customer ID,1.000000,0.028031,0.009918,-0.040094,0.007932,-0.005308
Recency,0.028031,1.000000,-0.214235,-0.227478,-0.321802,-0.103890
Frequency,0.009918,-0.214235,1.000000,0.788310,0.867775,-0.081312
Monetary,-0.040094,-0.227478,0.788310,1.000000,0.717126,-0.065134
Variety,0.007932,-0.321802,0.867775,0.717126,1.000000,-0.126965
Duration,-0.005308,-0.103890,-0.081312,-0.065134,-0.126965,1.000000


## Scaling

In [ ]:
scaler = StandardScaler()
rfmvd_tmp = rfmvd[['Recency', 'Frequency', 'Monetary', 'Variety', 'Duration']]
rfmvd_scaled = pd.DataFrame(scaler.fit_transform(rfmvd_tmp))
rfmvd_scaled.columns =['Recency', 'Frequency', 'Monetary', 'Variety', 'Duration']
rfmvd_scaled.head(1)

,Recency,Frequency,Monetary,Variety,Duration
0,1.569937,-0.281706,-0.309073,-0.459605,0.073869
